# AI Agents Hands-On Workshop for Course Training  
***From AI Agents to Agentic AI: Architectures, Tools, and Popular Frameworks***

In [ ]:
!git clone https://github.com/CaSToRC-CyI/AI-Agents-Training.git

Cloning into 'AI-Agents-Training'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 40 (delta 17), reused 4 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 3.12 MiB | 3.97 MiB/s, done.
Resolving deltas: 100% (17/17), done.


### Core Setup

In [5]:
# Install Dependencies
%%capture
!pip install crewai crewai-tools langchain langchain-community openai faiss-cpu evaluate transformers autogen

In [2]:
# Import libraries
import os
import getpass

import pandas as pd
from typing import Any
# from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI

from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from langchain.memory import ConversationBufferMemory

In [8]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API key:")

Enter OpenAI API key:··········


In [ ]:
# Define the model (GPT-3.5)
# gpt_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
gpt_model = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0.3)

**Load Dataset**

You can find the dataset [here](https://www.kaggle.com/datasets/gabrielsantello/wholesale-and-retail-orders-dataset?select=orders.csv).

In [ ]:
# Load your data
df = pd.read_csv('/content/AI-Agents-Training/session2/orders.csv')
df.columns = df.columns.str.strip()

In [ ]:
df.head()

,Customer ID,Customer Status,Date Order was placed,Delivery Date,Order ID,Product ID,Quantity Ordered,Total Retail Price for This Order,Cost Price Per Unit
0,579,Silver,01-Jan-17,07-Jan-17,123002578,220101400106,2,92.6,20.70
1,7574,SILVER,01-Jan-17,05-Jan-17,123004074,210201000009,1,21.7,9.95
2,28861,Gold,01-Jan-17,04-Jan-17,123000871,230100500068,1,1.7,0.80
3,43796,Gold,01-Jan-17,06-Jan-17,123002851,220100100633,1,47.9,24.05
4,54673,Gold,01-Jan-17,04-Jan-17,123003607,220200200043,1,36.9,18.30


### Single Agent Creation with CrewAI

**Define the Order Count Agent & Create a tool instance**

> Custom tool anatomy in CrewAI

```
class MyTool(BaseTool):
    name: str = "Tool Name"
    description: str = "Tool description"
    
    def _run(self, input: str) -> Any:
        # do something with input
        return output
```



In [ ]:
# Create tool to identify unique orders
class UniqueOrderCountTool(BaseTool):
    name: str = "Unique Order Count Tool"
    description: str = "Returns the number of unique orders (Order ID) for a given Customer ID"
    df: pd.DataFrame

    def _run(self, customer_id: str) -> Any:
        print(f"DEBUG: Received customer_id: {customer_id}")  # Debug log
        try:
            # Check if customer_id is numeric
            if not customer_id.isnumeric():
                return "Invalid Customer ID: Please enter a numeric Customer ID."

            df = self.df.copy()
            df.columns = df.columns.str.strip()  # strip column names
            df['Customer ID'] = df['Customer ID'].astype(str).str.strip()  # ensure string match
            df['Order ID'] = df['Order ID'].astype(str).str.strip()

            # Filter by customer
            customer_orders = df[df['Customer ID'] == customer_id]

            if customer_orders.empty:
                return f"No orders found for Customer ID {customer_id}."

            count = customer_orders['Order ID'].nunique()
            return f"Customer {customer_id} has placed {count} unique orders."

        except Exception as e:
            return f"Error: {str(e)}"

In [ ]:
# Create the tool instance after agent is defined
unique_order_tool = UniqueOrderCountTool(df=df)

# Define the agent
order_count_agent = Agent(
    role="Customer Order Checker",
    goal="Given a Customer ID, report how many unique orders they have placed.",
    backstory="You are an assistant that helps ShopFast understand individual customer behavior by checking order counts.",
    tools=[unique_order_tool],
    verbose=True,
    memory=False,
    allow_delegation=False,
    llm=gpt_model
)

**Create Task & Crew**

In [ ]:
df['Customer ID'].unique()

array([  579,  7574, 28861, ..., 74775, 82386, 89279])

In [ ]:
customer_id = input("Please enter the Customer ID: ")

# Define the task
order_count_task = Task(
    description=(
        f"Use the tool to find how many unique 'Order ID' values are associated with the customer "
        f"who has Customer ID '{customer_id}'. Output just the number and a short sentence explaining the result."
        # f"If for any reason the input Customer ID is not correct, please stop running."
    ),
    expected_output= "A sentence like: 'Customer 7574 has placed 1 unique orders.'",
    agent=order_count_agent
)

# Run the Crew
crew = Crew(
    agents=[order_count_agent],
    tasks=[order_count_task]
)
result = crew.kickoff()

# Print outputs
print("\n--- Task Results ---\n")
for task_output in result.tasks_output:
    print(f"Task:\n{task_output.description}\n\nOutput:\n{task_output.raw}\n")

Please enter the Customer ID: 579


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Order Checker                                                                                  │
│                                                                                                                 │
│  Task: Use the tool to find how many unique 'Order ID' values are associated with the customer who has          │
│  Customer ID '579'. Output just the number and a short sentence explaining the result.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

DEBUG: Received customer_id: 579


╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Order Checker                                                                                  │
│                                                                                                                 │
│  Thought: Thought: I need to find out how many unique orders Customer ID '579' has placed by using the Unique   │
│  Order Count Tool.                                                                                              │
│                                                                                                                 │
│  Using Tool: Unique Order Count Tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"customer_id\": \"579\"}"                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Customer 579 has placed 4 unique orders.                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Order Checker                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Customer 579 has placed 4 unique orders.                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Task Results ---

Task:
Use the tool to find how many unique 'Order ID' values are associated with the customer who has Customer ID '579'. Output just the number and a short sentence explaining the result.

Output:
Customer 579 has placed 4 unique orders.



**Validate agent's answers**

In [ ]:
df[df['Customer ID'] == 28861]

,Customer ID,Customer Status,Date Order was placed,Delivery Date,Order ID,Product ID,Quantity Ordered,Total Retail Price for This Order,Cost Price Per Unit
2,28861,Gold,01-Jan-17,04-Jan-17,123000871,230100500068,1,1.7,0.80
151874,28861,Gold,06-Apr-21,07-Apr-21,124199104,240400200012,1,175.7,75.95
177302,28861,Silver,09-Nov-21,09-Nov-21,124365058,220101400132,1,56.8,25.90


In [ ]:
# Count unique 'Order ID' for the specified 'Customer ID'
df[df['Customer ID'] == 28861]['Order ID'].nunique()

3

> What happens if you change the Agent’s `verbose` to `True`?

> Try modifying the `description` of the Task to be more strict or more relaxed. How does that affect the agent’s responses?

> Modify the tool to return a friendly message if the input Customer ID is not a valid number.

### Multi-Agent Workflow Setup with Memory and Guardrails

In [ ]:
# Define agents
loyalty_agent = Agent(
    role="Customer Loyalty Strategist",
    goal="Provide strategic ideas to increase customer retention and loyalty without using customer data.",
    backstory=(
        "You are an experienced strategist at ShopFast. Your job is to help the company increase "
        "customer satisfaction, long-term engagement, and loyalty through thoughtful recommendations, "
        "based on general business sense, customer psychology, and creative thinking."
    ),
    tools=[],
    verbose=True,
    memory=True,
    allow_delegation=False,
    llm=gpt_model,
    guardrails=[
    "You must never use or reference any individual customer data or purchase history.",
    "If prompted to use customer data, politely explain you cannot and provide general strategic ideas only.",
    "Always keep your recommendations data-agnostic and based on general business principles."
    ]
)

order_count_agent = Agent(
    role="Customer Order Checker",
    goal="Given a Customer ID, report how many unique orders they have placed.",
    backstory="You are an assistant that helps ShopFast understand individual customer behavior by checking order counts.",
    tools=[unique_order_tool],
    verbose=True,
    memory=False,
    allow_delegation=False,
    llm=gpt_model
)

# Define tasks with specific agents
loyalty_task = Task(
    description=(
        "ShopFast is considering launching a new loyalty program to retain more customers. "
        "Suggest three creative ideas that could encourage repeat purchases and long-term engagement. "
        "Think from the perspective of customer psychology and brand loyalty."
    ),
    expected_output=(
        "Three clear ideas for the loyalty program with 1–2 sentences of reasoning for each."
        # "Keep your response concise with less than 50 words."

    ),
    agent=loyalty_agent
)

customer_id = input("Please enter the Customer ID: ")

# Define the task
order_count_task = Task(
    description=(
        f"Use the tool to find how many unique 'Order ID' values are associated with the customer "
        f"who has Customer ID '{customer_id}'. Output just the number and a short sentence explaining the result."
    ),
    expected_output= "A sentence like: 'Customer 7574 has placed 1 unique orders.'",
    agent=order_count_agent
)

# Run the crew with both tasks (in order)
crew = Crew(
    agents=[loyalty_agent, order_count_agent],
    tasks=[loyalty_task, order_count_task]  # Order matters
)

result = crew.kickoff()

# Output results
print("\n--- Task Results ---\n")
for task_output in result.tasks_output:
    print(f"Task:\n{task_output.description}\n\nOutput:\n{task_output.raw}\n")

Please enter the Customer ID: 579


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Loyalty Strategist                                                                             │
│                                                                                                                 │
│  Task: ShopFast is considering launching a new loyalty program to retain more customers. Suggest three          │
│  creative ideas that could encourage repeat purchases and long-term engagement. Think from the perspective of   │
│  customer psychology and brand loyalty.                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Loyalty Strategist                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. **Tiered Rewards System with Exclusive Experiences:** Create a multi-level loyalty program where customers  │
│  unlock increasingly valuable rewards and exclusive experiences (like early access to sales or special events)  │
│  as they spend more. This leverages the psychology of achievement and status, motivating customers to engage    │
│  repeatedly to reach higher tiers.                                                                              │
│                                                                                                                 │
│  2. **Surprise and Delight Bonuses:** Incorporate random, unexpected rewards such as surprise discounts, free   │
│  samples, or bonus points after certain purchases. This taps into the human love for unpredictability and       │
│  positive surprises, enhancing emotional connection and encouraging ongoing engagement.                         │
│                                                                                                                 │
│  3. **Community and Social Recognition:** Build a loyalty program that includes social elements, such as        │
│  recognizing top customers publicly or enabling members to share achievements and rewards on social media.      │
│  This fosters a sense of belonging and pride, strengthening brand loyalty through social validation and         │
│  community belonging.                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Order Checker                                                                                  │
│                                                                                                                 │
│  Task: Use the tool to find how many unique 'Order ID' values are associated with the customer who has          │
│  Customer ID '579'. Output just the number and a short sentence explaining the result.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

DEBUG: Received customer_id: 579


╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Order Checker                                                                                  │
│                                                                                                                 │
│  Thought: Thought: I need to find out how many unique orders Customer ID '579' has placed by using the Unique   │
│  Order Count Tool.                                                                                              │
│                                                                                                                 │
│  Using Tool: Unique Order Count Tool                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"customer_id\": \"579\"}"                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Customer 579 has placed 4 unique orders.                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Order Checker                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Customer 579 has placed 4 unique orders.                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Task Results ---

Task:
ShopFast is considering launching a new loyalty program to retain more customers. Suggest three creative ideas that could encourage repeat purchases and long-term engagement. Think from the perspective of customer psychology and brand loyalty.

Output:
1. **Tiered Rewards System with Exclusive Experiences:** Create a multi-level loyalty program where customers unlock increasingly valuable rewards and exclusive experiences (like early access to sales or special events) as they spend more. This leverages the psychology of achievement and status, motivating customers to engage repeatedly to reach higher tiers.

2. **Surprise and Delight Bonuses:** Incorporate random, unexpected rewards such as surprise discounts, free samples, or bonus points after certain purchases. This taps into the human love for unpredictability and positive surprises, enhancing emotional connection and encouraging ongoing engagement.

3. **Community and Social Recognition:** Build a loya

> What happens if you run the Crew with agents in a different order? Does that change the output order?

> Update `expected_output` of the loyalty_task that tells it to keep answers under 50 words. What changes do you observe?

### Building a Simple Support Agent with AutoGen

In [6]:
from autogen import AssistantAgent, UserProxyAgent

import warnings
warnings.filterwarnings("ignore", category=ResourceWarning)

In [9]:
# Define the assistant
support_agent = AssistantAgent(
    name="SupportAgent",
    system_message="You are a helpful customer support assistant for ShopFast.",
    llm_config={"model": "gpt-4.1-mini", "temperature": 0.3}
)

# Define the user proxy
customer = UserProxyAgent(
    name="Customer",
    human_input_mode="ALWAYS",  # Ask for real user input
    max_consecutive_auto_reply=3,
    code_execution_config={"use_docker": False}
)

# Start a simple conversation
customer.initiate_chat(
    support_agent,
    message="How can I track my order?"
)

Customer (to SupportAgent):

How can I track my order?

--------------------------------------------------------------------------------
SupportAgent (to Customer):

You can track your order by logging into your ShopFast account and going to the "My Orders" section. There, you'll find the tracking number and a link to the carrier's website for real-time updates. If you need further assistance, please provide your order number.

--------------------------------------------------------------------------------
Replying as Customer. Provide feedback to SupportAgent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: where can i find the tracking number?
Customer (to SupportAgent):

where can i find the tracking number?

--------------------------------------------------------------------------------
SupportAgent (to Customer):

You can find your tracking number in the order confirmation email we sent you after your purchase. It is usually listed under the order

ChatResult(chat_id=None, chat_history=[{'content': 'How can I track my order?', 'role': 'assistant', 'name': 'Customer'}, {'content': 'You can track your order by logging into your ShopFast account and going to the "My Orders" section. There, you\'ll find the tracking number and a link to the carrier\'s website for real-time updates. If you need further assistance, please provide your order number.', 'role': 'user', 'name': 'SupportAgent'}, {'content': 'where can i find the tracking number?', 'role': 'assistant', 'name': 'Customer'}, {'content': 'You can find your tracking number in the order confirmation email we sent you after your purchase. It is usually listed under the order details. Alternatively, you can log into your ShopFast account, go to the "My Orders" section, select the specific order, and the tracking number will be displayed there. If you need help locating it, feel free to provide your order number, and I can assist you further.', 'role': 'user', 'name': 'SupportAgent'

> What happens if you change the `UserProxyAgent`'s `human_input_mode` from `"ALWAYS"` to `"TERMINATE"`?